In [1]:
import sys
sys.path.append('../scripts')

import importlib

import model_training_multiclassificacao as mtm
import utils

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
importlib.reload(mtm)
importlib.reload(utils)

<module 'utils' from '/Users/patricia/Documents/code/python/behavior-detection/notebooks/../scripts/utils.py'>

In [3]:
X, y = mtm.load_data()
y.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5525 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.2+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 5525 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5525 non-null   object
dtypes: object(1)
memory usage: 43.3+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [4]:
X_train, X_test, y_train, y_test = mtm.split_train_test_data(X, y)


Tamanho do conjunto de treino: 3867
Tamanho do conjunto de teste: 1658


In [5]:
print(f"Tipo de X_train: {type(X_train)}")
print(f"Tipo de y_train: {type(y_train)}")
print(f"Tipo de X_test: {type(X_test)}")
print(f"Tipo de y_test: {type(y_test)}")

Tipo de X_train: <class 'pandas.core.frame.DataFrame'>
Tipo de y_train: <class 'pandas.core.series.Series'>
Tipo de X_test: <class 'pandas.core.frame.DataFrame'>
Tipo de y_test: <class 'pandas.core.series.Series'>


In [6]:
y_train = y_train.values
y_test = y_test.values

In [7]:
results = mtm.train_and_evaluate_models(X_train, X_test, y_train, y_test, enable_cv=False, enable_gridsearch=True, enable_randomsearch=False, enable_bayesian_optimization=False)

Executing pipeline for Logistic Regression com GridSearch ...

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been depre

/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patricia/anaconda3/envs/projetos_ML/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_